# GIS Practicum Project
____________________________________
## Impact of Extreme Weather Events on Railway Incidents
### Determining high risk locations where monitoring stations could be installed

Data Sources:

    1. NOAA Reference Data:
        - source: ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/
        - local files: ghcnd-stations.txt & ghcnd-states.txt
        - use: reference data for daily weather station files
    2. NOAA Daily Weather Data:
        - source: ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/
        - local files: ghcnd_hcn.tar.gz
        - use: granular weather data by station
    3. U.S. Cities Data:
        - source: https://github.com/kelvins/US-Cities-Database/blob/main/csv/us_cities.csv
        - local files: us_cities.csv
        - use: mapping between datasets
    4. Railroad Grade Crossing Incident Data:
        - source: https://data.transportation.gov/Railroads/Highway-Rail-Grade-Crossing-Accident-Data/7wn6-i5b9
        - local files: Highway-Rail_Grade_Crossing_Accident_Data.csv
        - use: main incident dataset
    5. Weather Events:
        - source: https://www.kaggle.com/sobhanmoosavi/us-weather-events
        - local file: WeatherEvents_Jan2016-Dec2020.csv
        - use: major U.S. weather events, 6.3m over 5yr period
    
Outline:
    1. Environment setup
    2. Data loading & Preprocessing
    3. EDA & Visualization
    4. Prep for scikit learn
    5. Model Building & Evaluation

## 1. Environment Setup
___

In [ ]:
import csv
import glob
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import seaborn as sns

Set Matplotlib Style:

In [ ]:
plt.style.use('dark_background')

Get Current Working Directory:

In [ ]:
wd = os.getcwd()
print(wd)

Establish Constants:

In [ ]:
# define relevant years list for treatments
relevant_years_list = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [ ]:
# assign a random seed
np.random.seed(2021)

## 2. Data Loading & Preprocessing
___

### Source 1: NOAA Reference Data:

#### Stations:

In [ ]:
def load_noaa_stations_data(wd):
    """
    This function will load the weather station dataset from NOAA
    this is used for mapping to rail crossing locations.
    
    Contains weather station reference data
    
    Source: ghcnd-stations.txt from NOAA ftp
    Input: wd - working directory
    Output: stations_df - dataframe of NOAA stations
    """
    f = open(os.path.join(wd,"ghcnd-stations.txt"),"r")
    lines = f.readlines()

    # columns in the station file
    colnames = ['ID', 'LAT', 'LON', 'ELEV', 'STATE', 'NAME', 'GSN', 'HCNCRN', 'WMOID']
    stationlist = []

    # initialize dataframe with correct columns
    stations_df = pd.DataFrame(columns=colnames)

    # iterate through stations and add them to our collection of stations if they are in the US
    for line in lines:
        # first 2 characters are the country code , we only care about us stations
        if line[0:2] == 'US':

            # the description of the file seemed slightly off, i tested and found these column numbers to work best
            row = {"ID": line[0:11].upper(),
                    "LAT": float(line[13:20]),
                    "LON": float(line[21:30]),
                    "ELEV": float(line[31:37]),
                    "STATE": line[38:40],
                    "NAME": line[41:71],
                    "GSN": line[72:75],
                    "HCNCRN": line[76:79],
                    "WMOID": line[80:85]
                   }
            stationlist.append(row)
        else:
            pass
    stations_df = stations_df.append(stationlist)
    f.close()
    
    return stations_df

In [ ]:
stations_df = load_noaa_stations_data(wd)

In [ ]:
stations_df.head()

#### States:

In [ ]:
def load_noaa_states_data(wd):
    """
    This function will load the state dataset from NOAA
    this is used for mapping to rail crossing locations.
    
    Contains state reference data
    
    Source: ghcnd-states.txt from NOAA ftp
    Input: wd - working directory
    Output: states_df - dataframe of NOAA states
    """
    # read in states dataset to supplement weather stations data
    f = open(os.path.join(wd,"ghcnd-states.txt"),"r")
    lines = f.readlines()

    colnames = ['CODE', 'NAME']

    # create dataframe of state data
    states_df = pd.DataFrame(columns=colnames)
    for line in lines:
        modline = line.strip('\n')
        data = {'CODE': line[0:2],
                "NAME": modline[3:50]
               }
        states_df = states_df.append(data, ignore_index=True)    

    f.close()
    return states_df

In [ ]:
states_df = load_noaa_states_data(wd)

In [ ]:
states_df.head()

#### NOAA Reference Data Merge:

In [ ]:
def merge_noaa_refdata(stations_df, states_df):
    """
    This function will merge the NOAA refdata
    this is used for mapping to rail crossing locations.
    
    Contains state & station reference data
    
    Input: stations_df, states_df
    Output: stations_plus_df - dataframe of NOAA refdata
    """
    
    # add state data to the stations dataset
    station_plus_df = stations_df.join(states_df.set_index('CODE'), on='STATE', rsuffix='_STATE')

    # create our key feature: coordinateID (wcoordinateID for weather)
    # round latitude & longitude to 1 decimal, combine them in a tuple (lat, lon)
    station_plus_df['wcoordinateID'] = list(zip(round(station_plus_df['LAT'],1),round(station_plus_df['LON'],1)))
    station_plus_df = station_plus_df[['ID','ELEV','wcoordinateID']]
    
    return station_plus_df

In [ ]:
station_plus_df = merge_noaa_refdata(stations_df, states_df)

In [ ]:
station_plus_df.head()

### Source 3: U.S. Cities:

In [ ]:
def load_us_cities_data(wd):
    """
    This function will load cities data which will 
    be used to attach coordinateID to other datasets 
    which only have city or county level data.
    Also derives county locations.
    
    Input: wd - working directory
    Output: grouped_meancounties_df
    """
    cities_df = pd.read_csv(os.path.join(wd,"us_cities.csv"))
    
    # standardize county and state, city is not populated for all events.
    # one change to approach would be to include all cities + the grouped mean of each county
    cities_df['County'] = cities_df['COUNTY'].str.upper()
    cities_df['State'] = cities_df['STATE_NAME'].str.upper()

    # subset of data that we care about, lat+lon to make coordinateID, county, state, state code to merge on
    counties = cities_df[['County','State','LATITUDE','LONGITUDE','STATE_CODE']]
    grouped_counties = counties.groupby(['State','County'])
    grouped_meancounties_df = grouped_counties.mean()
    grouped_meancounties_df = grouped_meancounties_df.reset_index()
    grouped_meancounties_df['wcoordinateID'] = list(zip(round(grouped_meancounties_df['LATITUDE'],1),round(grouped_meancounties_df['LONGITUDE'],1)))
    
    return grouped_meancounties_df

In [ ]:
grouped_meancounties_df = load_us_cities_data(wd)

In [ ]:
grouped_meancounties_df.head()

### Source 4: Rail Crossing Data:

In [ ]:
def load_rail_crossing_data(wd, grouped_meancounties_df):
    """
    This function will load data for rail crossings
    which will be used for instances for model training.
    Will also be used to limit weather station observations
    
    Input: wd - working directory, grouped_meancounties_df - location base data
    Output: rail_city_df
    """
    railcrossing_df = pd.read_csv(os.path.join(wd,"Highway-Rail_Grade_Crossing_Accident_Data.csv"))
    
    # gather the fields necessary for coordinateID, as well as any  fields you want for analysis later 
    # change to approache what fields we include in refined_rr
    refined_rr_df = railcrossing_df #[['Incident Number','Date','County Name', 'State Name']]

    # drop any incident without a date
    refined_rr_df = refined_rr_df.dropna(subset=['Date'])

    # create our feature incident date, which is an integer with format: yyyymmdd 
    incident_date = refined_rr_df['Date'].str.split(' ', expand=True)
    incident_date = incident_date[0].str.split('/', expand=True)
    refined_rr_df['incident_date'] = (incident_date[2].astype(int) * 10000) + (incident_date[0].astype(int) * 100) + (incident_date[1].astype(int) * 1)

    # merge accident data with city/county data to add coordinateID to each accident.
    merg_rail_city_df = refined_rr_df.merge(grouped_meancounties_df, how='inner', left_on=['County Name','State Name'], right_on=['County','State'])
    print("Shape of merged unfiltered rail_city dataset:  {}".format(merg_rail_city_df.shape))
    merg_rail_city_df = merg_rail_city_df[merg_rail_city_df['incident_date'] > 20140000]
    merg_rail_city_df = merg_rail_city_df[['Grade Crossing ID', 
                                           'Maintenance Parent Railroad Code', 
                                           'Incident Number',
                                           'Crossing Illuminated',
                                           'Railroad Type',
                                           'Track Type Code', 
                                           'incident_date', 
                                           'wcoordinateID', 
                                           'State', 
                                           'County'
                                          ]]
    print("Shape of merged filtered rail_city dataset:  {}".format(merg_rail_city_df.shape))
    
    return merg_rail_city_df

In [ ]:
merg_rail_city_df = load_rail_crossing_data(wd, grouped_meancounties_df)

In [ ]:
def pre_process_rail(merg_rail_city_df):
    """
    This function will preprocess rail df.
    Focus is on datetime formats
    
    input: merg_rail_city_df
    output: enriched merg_rail_city_df
    """
    
    merg_rail_city_df['incident_datetime'] = pd.to_datetime(merg_rail_city_df['incident_date'], format='%Y%m%d')
    merg_rail_city_df['incident_year'] = merg_rail_city_df['incident_datetime'].dt.year
    merg_rail_city_df['incident_month'] = merg_rail_city_df['incident_datetime'].dt.month
    merg_rail_city_df['incident_year_month'] = merg_rail_city_df['incident_year'].astype(str) + '_' + merg_rail_city_df['incident_month'].astype(str)
    
    return merg_rail_city_df

In [ ]:
merg_rail_city_df = pre_process_rail(merg_rail_city_df)

In [ ]:
merg_rail_city_df.head()

#### Filter Weather Stations:

In [ ]:
def filter_weather_stations(station_plus_df, merg_rail_city_df, relevant_years_list):
    """
    This function will filter for relevant weather stations
    
    input: 
    - station_plus_df (enriched station data)
    - merg_rail_city_df (rail incident & city data)
    - relevant_years_list
    output: 
    - incident_stations list
    - merged_stations_incidents_df (filtered data for location and years)
    """

    # dataframe of weather stations - only those that share coordinateID with an accident
    merged_stations_incidents_df = station_plus_df.merge(merg_rail_city_df,left_on='wcoordinateID', right_on='wcoordinateID', how='inner')

    # filter by state
    target_states = ['NEW JERSEY','NEW YORK','PENNSYLVANIA','CONNECTICUT','DELAWARE','MARYLAND','MASSACHUSETTS','NEW HAMPSHIRE','VIRGINIA']
    target_state_codes = ['NJ', 'NY', 'PA', 'CT', 'DE', 'MD', 'MA', 'NH', 'VA']
    statefiltered_stations_incidents_df = merged_stations_incidents_df[merged_stations_incidents_df['State'].isin(target_states)]

    # filter by year
    yearstatefiltered_stations_incidents_df = statefiltered_stations_incidents_df[statefiltered_stations_incidents_df['incident_year'].isin(relevant_years_list)]

    # save a list of the station IDs that were included in the merged dataframe
    incident_stations = [x.upper() for x in yearstatefiltered_stations_incidents_df['ID'].unique()]

    print(len(incident_stations))
    
    return incident_stations, merged_stations_incidents_df

In [ ]:
incident_stations, merged_stations_incidents_df = filter_weather_stations(station_plus_df, merg_rail_city_df, relevant_years_list)

In [ ]:
merged_stations_incidents_df.head()

In [ ]:
merged_stations_incidents_df.shape

### Source 2: NOAA Daily Weather Data:

#### NOTE:  Source 2 needed to be loaded here in order to use the incident_stations list to filter which stations will be captured, this was done to reduce preprocessing effort

### Instructions

#### IMPORTANT - You will need ~40 GB of HDD or SSD space for NOAA Data, as well as time (~2hrs depending on connection quality) for downloading + extracting noaa data 

1. Download all files  and move them to the directory you plan to work in (working directory / wd)
2. Select the file ghcnd_all.tar.gz and open it with your unzipping tool (ie: WinRAR), it will take time to load due to the size (~120,000 files)
3. Upon Completion, you will see a text file ghcnd-version.txt, and folder ghcnd_all.  

Two options for the next part:
1. Extracting all data
2. Extracting only US data

Both take time, only US data is a slightly faster but more effort up front, each option is described below.

1. Extracting all data:
- Select the folder ghcnd_all and extract to your current working directory (wd).  You will need ~40 GB  of space to be safe.  Extraction will take a while. 

2. Extracting only US data:
- Select the folder ghcnd_all and open it in WinRAR/7Zip, it will take a minute to load all the files inside.  Sort the files by name. Scroll down to the prefix 'US'. Select all files beginning with 'US'. and extract them to a new folder 'ghcnd_all' in your working directory (wd/ghcnd_all). Extraction still takes a while, but this is faster than getting all 120k files.

In [ ]:
def load_noaa_dailies(wd):
    '''
    This function will load the noaa daily weather station data
    this data contains the values associated w station ref data
    
    prior attempts to include this data failed because ghcnd_ghc.tar.gz is too large.
    by limiting the number of stations included to only those where incidents occurred,
    and by limiting the observation years from each station, we can reduce the amount of
    memory required to process this data
    
    input: wd (your working directory, assuming above instructions followed)
    output: noaa_relevant_dailies.csv (written to your wd)
    '''
    # with assistance from 
    # https://stackoverflow.com/questions/62165172/convert-dly-files-to-csv-using-python
    # fields as given by the spec
    
    fields = [
        ["ID", 1, 11],
        ["YEAR", 12, 15],
        ["MONTH", 16, 17],
        ["ELEMENT", 18, 21]]

    offset = 22

    for value in range(1, 32):
        fields.append((f"VALUE{value}", offset,     offset + 4))
        fields.append((f"MFLAG{value}", offset + 5, offset + 5))
        fields.append((f"QFLAG{value}", offset + 6, offset + 6))
        fields.append((f"SFLAG{value}", offset + 7, offset + 7))
        offset += 8

    # Modify fields to use Python numbering
    fields = [[var, start - 1, end] for var, start, end in fields]
    fieldnames = [var for var, start, end in fields]


    # the goal of this code is to make 1 file TOTAL from many (originally 1 per station)

    # enter where you want a csv saved - it will be many Gigs
    csv_filename = wd+'\\noaa_relevant_dailies.csv'

    with open(csv_filename, 'w', newline='') as f_csv:

        # glob.glob should aim at the folder where you extracted all the daily files, wd/ghcnd_all - do not forget to include '\*.dly'
        for dly_filename in glob.glob(r'C:\Users\thoma\Desktop\Rutgers MBS\Externship Class\Practicum\Project\ghcnd_all\*.dly', recursive=True):
            path, name = os.path.split(dly_filename)
            station = name[:-4].upper()
            if station in incident_stations:
                # you could replace this with adding to a dataframe or something else, but i am running out of brain power.
                with open(dly_filename, newline='') as f_dly:
                    spamwriter  = csv.writer(f_csv)
                    spamwriter.writerow(fieldnames) 

                    for line in f_dly:
                        row = [line[start:end].strip() for var, start, end in fields]
                        year = int(row[1])

                        # important check to save memory, only add recent observations
                        if year > 2014:
                            spamwriter.writerow(row)

In [ ]:
load_noaa_dailies(wd)

### Preprocess NOAA Daily Data

In [ ]:
def clean_noaa_dailies(wd):
    """
    This function will clean the noaa_relevant_dailies data
    
    input: wd (working directory, will find relevant file)
    output: final_daily_observations.csv (cleaned data)
    """
    # load data written from previous function
    df = pd.read_csv(os.path.join(wd,"noaa_relevant_dailies.csv"))
    # we added a header row for every file, but we only need 1 header row. remove the others:
    df = df[df['YEAR'] != 'YEAR']

    # month and year had some strings and some ints. lets standardize
    df['YEAR'] = pd.to_numeric(df['YEAR'])
    df['MONTH'] = pd.to_numeric(df['MONTH'])


    # base for transposed data
    base = pd.DataFrame(columns=['ID','YEAR','MONTH','ELEMENT','VALUE', 'MFLAG', 'QFLAG', 'SFLAG'])

    # loop through all days to partially transpose the file (day cols -> rows)
    for i in range(1,32):
        colnames = [f'VALUE{i}', f'MFLAG{i}', f'QFLAG{i}', f'SFLAG{i}']
        newcolnames = ['VALUE', 'MFLAG', 'QFLAG', 'SFLAG']
        col_order = ['ID','YEAR','MONTH','DAY','ELEMENT', colnames[0], colnames[1], colnames[2], colnames[3]]

        df_new = df[['ID','YEAR','MONTH','ELEMENT', colnames[0], colnames[1], colnames[2], colnames[3]]]
        df_new['DAY'] = i
        df_new = df_new[col_order]
        df_new = df_new.rename(columns={colnames[0]:newcolnames[0], colnames[1]:newcolnames[1], colnames[2]:newcolnames[2], colnames[3]:newcolnames[3]})
        base = pd.concat([base, df_new], sort=False)


    newcsv = base[['ID','YEAR','MONTH','DAY','ELEMENT','VALUE','MFLAG','QFLAG','SFLAG']]

    daily_station_coordinates = station_plus_df[['ID','wcoordinateID']]
    daily_final = newcsv.merge(daily_station_coordinates, left_on='ID', right_on='ID')
    daily_final.to_csv('final_daily_observations.csv')
    print('cleaning complete.  final shape: ', daily_final.shape, '.  reload directory to see file')

In [ ]:
clean_noaa_dailies(wd)

### Derived NOAA Features:

In [ ]:
noaa_dailies_df = pd.read_csv(os.path.join(wd,'final_daily_observations.csv'))

In [ ]:
noaa_dailies_df.ID.nunique()

In [ ]:
noaa_dailies_df.head().drop('Unnamed: 0', 1)

In [ ]:
def preprocess_noaa_dailies(noaa_dailies_df):
    """
    This function will filter the noaa dailies data
    and derive a set of aggregate features for our model
    
    input: noaa_dailies_df (loaded and filtered)
    output: noaa_station_aggs_df (features)
    """
    # filter for non-error (-9999 indicates measurement error) and for relevant date range
    noaa_dailies_df = noaa_dailies_df[(noaa_dailies_df['VALUE']!=-9999) & noaa_dailies_df['YEAR'].isin([2016, 2017, 2018, 2019, 2020, 2021])]
    # create year_month aggregator
    noaa_dailies_df['year_month'] = noaa_dailies_df['YEAR'].astype(str) + '_' + noaa_dailies_df['MONTH'].astype(str)
    
    # derive features
    # max
    MDP_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='PRCP'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].max().to_frame().rename(columns={'VALUE':'Max Daily Precipitation'}).reset_index()
    SNW_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='SNOW'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].max().to_frame().rename(columns={'VALUE':'Max Daily Snow'}).reset_index()
    SDP_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='SNWD'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].max().to_frame().rename(columns={'VALUE':'Max Daily Snow Depth'}).reset_index()
    TMX_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='TMAX'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].max().to_frame().rename(columns={'VALUE':'Max Daily TempMax'}).reset_index()
    TMN_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='TMIN'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].max().to_frame().rename(columns={'VALUE':'Max Daily TempMin'}).reset_index()
    # min
    MDP1_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='PRCP'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].min().to_frame().rename(columns={'VALUE':'Min Daily Precipitation'}).reset_index()
    SNW1_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='SNOW'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].min().to_frame().rename(columns={'VALUE':'Min Daily Snow'}).reset_index()
    SDP1_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='SNWD'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].min().to_frame().rename(columns={'VALUE':'Min Daily Snow Depth'}).reset_index()
    TMX1_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='TMAX'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].min().to_frame().rename(columns={'VALUE':'Min Daily TempMax'}).reset_index()
    TMN1_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='TMIN'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].min().to_frame().rename(columns={'VALUE':'Min Daily TempMin'}).reset_index()
    # mean
    MDP2_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='PRCP'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].mean().to_frame().rename(columns={'VALUE':'Mean Daily Precipitation'}).reset_index()
    SNW2_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='SNOW'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].mean().to_frame().rename(columns={'VALUE':'Mean Daily Snow'}).reset_index()
    SDP2_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='SNWD'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].mean().to_frame().rename(columns={'VALUE':'Mean Daily Snow Depth'}).reset_index()
    TMX2_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='TMAX'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].mean().to_frame().rename(columns={'VALUE':'Mean Daily TempMax'}).reset_index()
    TMN2_df = noaa_dailies_df[noaa_dailies_df['ELEMENT']=='TMIN'].groupby(['ID','wcoordinateID', 'year_month'])['VALUE'].mean().to_frame().rename(columns={'VALUE':'Mean Daily TempMin'}).reset_index()
    
    # combine data
    noaa_station_aggs_df = MDP_df.merge(SNW_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(SDP_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(TMX_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(TMN_df, how='outer', on=['ID','wcoordinateID', 'year_month'])

    noaa_station_aggs_df = noaa_station_aggs_df.merge(MDP1_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(SNW1_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(SDP1_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(TMX1_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(TMN1_df, how='outer', on=['ID','wcoordinateID', 'year_month'])

    noaa_station_aggs_df = noaa_station_aggs_df.merge(MDP2_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(SNW2_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(SDP2_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(TMX2_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    noaa_station_aggs_df = noaa_station_aggs_df.merge(TMN2_df, how='outer', on=['ID','wcoordinateID', 'year_month'])
    
    #rename columns for merging purposes
    noaa_station_aggs_df.rename(columns={'year_month':'incident_year_month'}, inplace=True)
    
    return noaa_station_aggs_df

In [ ]:
noaa_station_aggs_df = preprocess_noaa_dailies(noaa_dailies_df)

In [ ]:
noaa_station_aggs_df.head()

In [ ]:
noaa_station_aggs_df.shape

### Derive Truth Data and Baseline Features:

#### Get Incident Count Agg:

In [ ]:
def derive_inc_count(merged_stations_incidents_df):
    """
    This function will derive the incident counts,
    aggregate by location and year_month,
    and split out reference data
    
    input: merged_stations_incidents_df (incident data)
    output: incidents_df & ref_df 
    (incident counts and station_crossing ref data respectively)
    """
    # get aggregated incident data
    inc_counts_df = merged_stations_incidents_df.groupby(['wcoordinateID', 'incident_year_month'])['Incident Number'].count().to_frame().reset_index().rename(columns={'Incident Number': 'Incident Count'})
    fin_df = merged_stations_incidents_df.merge(inc_counts_df, how="left", on=['wcoordinateID', 'incident_year_month'])
    fin_df['Incident Count'].fillna(0, inplace=True)
    
    # create station_crossing reference data df
    ref_df = fin_df[['wcoordinateID',
                'ID',
                'Grade Crossing ID',
                 'ELEV',
                 'Maintenance Parent Railroad Code',
                 'Crossing Illuminated',
                 'Railroad Type',
                 'Track Type Code',
                 'State',
                 'County'
                ]]
    ref_df.drop_duplicates(inplace=True)
    
    # create incidents df for truth data derivation
    incidents_df = fin_df[['wcoordinateID',
                           'ID',
                           'Grade Crossing ID',
                           'incident_year',
                           'incident_year_month',
                           'Incident Count']]
    incidents_df.drop_duplicates(inplace=True)
    
    return incidents_df, ref_df

In [ ]:
incidents_df, ref_df = derive_inc_count(merged_stations_incidents_df)

In [ ]:
incidents_df.head()

In [ ]:
incidents_df.shape

In [ ]:
ref_df.head()

In [ ]:
ref_df.shape

### Assemble Final Feature Set:

In [ ]:
noaa_station_aggs_df.shape

In [ ]:
filtered_noaa = noaa_station_aggs_df[noaa_station_aggs_df['ID'].isin(ref_df['ID'])]

In [ ]:
filtered_noaa.shape

In [ ]:
feature_set_df = filtered_noaa.merge(ref_df[['ID','ELEV', 'Maintenance Parent Railroad Code', 'Railroad Type', 'Track Type Code', 'State', 'County']], how='left', on='ID')

In [ ]:
feature_set_df.head()

In [ ]:
feature_set_df.shape

In [ ]:
feature_set_df[feature_set_df['ID'].isin(incidents_df['ID'])].shape

In [ ]:
feature_set_w_truth_df = feature_set_df.merge(incidents_df[['ID', 'incident_year_month', 'Incident Count']], how='left', on=['ID', 'incident_year_month'])

In [ ]:
feature_set_w_truth_df.info()

In [ ]:
feature_set_w_truth_df['Incident Count'].fillna(0, inplace=True)

In [ ]:
feature_set_w_truth_df.info()

In [ ]:
def assign_risk_class(x):
    """Assigns a risk_class label."""
    x = int(x)
    if x > 6:
        label = 'High'
    elif x > 0 & x <= 6:
        label = 'Medium'
    else:
        label = 'Low'
        
    return label

In [ ]:
feature_set_w_truth_df['Risk_Level'] = feature_set_w_truth_df['Incident Count'].apply(lambda x: assign_risk_class(x))

In [ ]:
feature_set_w_truth_df.shape

In [ ]:
feature_set_w_truth_df.dropna(inplace=True)

In [ ]:
feature_set_w_truth_df.shape

## 3. EDA & Visualization:
___

In [ ]:
feature_set_w_truth_df.describe().round()

### Weather Station EDA & Visualization

In [ ]:
weather_station_crossings = merged_stations_incidents_df.groupby(['ID'])['Grade Crossing ID'].nunique().to_frame().sort_values('Grade Crossing ID', ascending=False)
weather_station_incidents = merged_stations_incidents_df.groupby(['ID'])['Incident Number'].nunique().to_frame().sort_values('Incident Number', ascending=False)
weather_station_stats = weather_station_crossings.merge(weather_station_incidents, how="outer", on="ID")
weather_station_stats.fillna(0, inplace=True)
weather_station_stats.head()

In [ ]:
weather_station_stats.shape

In [ ]:
weather_station_stats.mean()

In [ ]:
weather_station_stats.max()

In [ ]:
weather_station_stats.min()

In [ ]:
blue_circle = dict(markerfacecolor='cyan', marker='o')
mean_shape = dict(markerfacecolor='green', marker='D', markeredgecolor='green')

df = weather_station_stats
c = 'cyan'

fig, axs = plt.subplots(1, len(df.columns), figsize=(8,10))

for i, ax in enumerate(axs.flat):
    ax.boxplot(df.iloc[:,i], boxprops=dict(color=c), capprops=dict(color=c), whiskerprops=dict(color=c), medianprops=dict(color=c), flierprops=blue_circle, showmeans=True, meanprops=mean_shape, notch=True, vert=True)
    ax.set_title(df.columns[i], fontsize=20, fontweight='bold')
    ax.tick_params(axis='y', labelsize=14)
    ax.set_ylim(0, 75)
    
plt.tight_layout()
plt.show()

In [ ]:
feature_set_w_truth_df.head()

In [ ]:
feature_set_w_truth_df.shape

In [ ]:
categorical_features = [
                        'State',
                        'County',
                        'Maintenance Parent Railroad Code',
                        'Railroad Type', 
                        'Track Type Code',
                       ]

for feature in categorical_features:
    stats_df = feature_set_w_truth_df.groupby([feature])['ID'].count().reset_index().sort_values('ID', ascending=False).head(10)
    
    fig, ax = plt.subplots()
    x = np.arange(len(stats_df[feature]))
    width = 0.7

    rects1 = ax.bar(x, stats_df['ID'], width, label = 'Count of Instances')
    #rects2 = ax.bar(x + width/2, mode2_stats_df['Total Fatalities'], width, label = 'Incidents w Fatalities')

    ax.set_title('Instance Count by {} (Top 10 if >10)'.format(feature))
    ax.set_ylabel('Count of Instances')
    ax.set_xticks(x)
    ax.set_xticklabels(stats_df[feature])
    ax.legend()

    fig.tight_layout()

    plt.xticks(rotation=45)
    plt.show()

In [ ]:
stats_df = feature_set_w_truth_df.groupby(['Risk_Level'])['wcoordinateID'].count().reset_index().sort_values('wcoordinateID')

stats_df

In [ ]:
fig, ax = plt.subplots()
x = np.arange(len(stats_df['Risk_Level']))
width = 0.7

rects1 = ax.bar(x, stats_df['wcoordinateID'], width, label = 'Count of Crossings')
#rects2 = ax.bar(x + width/2, mode2_stats_df['Total Fatalities'], width, label = 'Incidents w Fatalities')

ax.set_title('Truth Data Labels for At-Grade Crossings')
ax.set_ylabel('Count of Crossings')
ax.set_xticks(x)
ax.set_xticklabels(stats_df['Risk_Level'])
ax.legend()

fig.tight_layout()

plt.show()

In [ ]:
feature_set_w_truth_df.ID.nunique()

In [ ]:
feature_set_w_truth_df.head()

In [ ]:
feature_set_w_truth_df.info()

In [ ]:
weather_stats_df = feature_set_w_truth_df.describe().round(2).T.head(16).reset_index()
weather_stats_df

In [ ]:
df = weather_stats_df


fig, ax = plt.subplots(figsize=(16,8)) 
scat = sns.scatterplot(data=df, x='mean', y='std', s=250*df['count']/(df['count'].max()), alpha=0.5, hue='index')

ax.set_ylabel('Features Standard Deviation (mm or tenths of degrees C)') 
ax.set_xlabel('Feature Mean (mm or tenths of degrees C)') 
ax.set_title('Weather Station Features') 

plt.grid(color='grey', linestyle = '--')
plt.ylim(0,300)

fig.tight_layout()
plt.show()

## 4. Prep for SciKitLearn:
____

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml, make_moons, make_circles, make_classification
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
COLOR = 'white'
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = ''
        else:
            title = ''

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

### Treatments:

#### Test 2021

In [ ]:
train_set_2021 = feature_set_w_truth_df[(feature_set_w_truth_df['incident_year_month'].str[:4]!='2021') & 
                                        (feature_set_w_truth_df['incident_year_month'].str[:4]!='2014') &
                                        (feature_set_w_truth_df['incident_year_month'].str[:4]!='2015')]
test_set_2021 = feature_set_w_truth_df[feature_set_w_truth_df['incident_year_month'].str[:4]=='2021']

In [ ]:
train_set_2021.shape

In [ ]:
test_set_2021.shape

#### Test 2020

In [ ]:
train_set_2020 = feature_set_w_truth_df[(feature_set_w_truth_df['incident_year_month'].str[:4]!='2021') & 
                                        (feature_set_w_truth_df['incident_year_month'].str[:4]!='2020') &
                                        (feature_set_w_truth_df['incident_year_month'].str[:4]!='2014')]
test_set_2020 = feature_set_w_truth_df[feature_set_w_truth_df['incident_year_month'].str[:4]=='2020']

In [ ]:
train_set_2020.shape

In [ ]:
test_set_2020.shape

#### Test 2019

In [ ]:
train_set_2019 = feature_set_w_truth_df[(feature_set_w_truth_df['incident_year_month'].str[:4]!='2021') & 
                                        (feature_set_w_truth_df['incident_year_month'].str[:4]!='2020') &
                                        (feature_set_w_truth_df['incident_year_month'].str[:4]!='2019')]
test_set_2019 = feature_set_w_truth_df[feature_set_w_truth_df['incident_year_month'].str[:4]=='2019']

In [ ]:
train_set_2019.shape

In [ ]:
test_set_2019.shape

## 5. Model Building and Evaluation:
___

## Baseline Models:

Define Key Information:

In [ ]:
# classes to predict
classes = ['Low', 'Medium', 'High']

# set data_df = dataset
data_df = feature_set_w_truth_df

# features definition
categorical_features = [
                        'State',
                        'County',
                        'Maintenance Parent Railroad Code',
                        'Railroad Type', 
                        'Track Type Code',
                       ]

numeric_features = [
#                    'ELEV',
#                    'Mean Daily Precipitation',
#                    'Mean Daily Snow',
#                    'Mean Daily Snow Depth',
#                    'Mean Daily TempMax',
#                    'Mean Daily TempMin',
#                    'Max Daily Precipitation',
#                    'Max Daily Snow',
#                    'Max Daily Snow Depth',
#                    'Max Daily TempMax',
#                    'Max Daily TempMin',
#                    'Min Daily Precipitation',
#                    'Min Daily Snow',
#                    'Min Daily Snow Depth',
#                    'Min Daily TempMax',
#                    'Min Daily TempMin',
                   ]

# X represents all feature columns
X = data_df.loc[:,(categorical_features + numeric_features)]
# note y should be a "Risk class" derived from # of incidents/injuries/fatalities somehow that matches 'classes' above
y = data_df['Risk_Level']

# classifiers to try
names = ["Decision Tree", "AdaBoost", "GradientBoost"]
classifiers = [DecisionTreeClassifier(max_depth=5),
               AdaBoostClassifier(),
               GradientBoostingClassifier(),
              ]

# training set ratios
tsizes = [0.7,0.8,0.9]

Transformation of Features:

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

### Multi Class Baseline (Train/Test/Split Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )
    
    # establish empty tsize lists
    tsize_precision = []
    tsize_recall = []
    
    for tsize in tsizes:
        # print tsize being tested
        print(f'test size: {tsize}')
        # train, test, split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
        # fit model
        clf.fit(X_train, y_train)
        # get confusion matrix stats
        y_pred = clf.predict(X_test)
        N = confusion_matrix(y_test, y_pred, labels=classes)
        # note that in the above N, rows are true label, column predicted label, values indicate # of samples
        # utilize plot confusion matrix function from earlier
        print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
        print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
        print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
        plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                          normalize=True,
                          title="Baseline TTS Treatment - {}".format(names[i]),
                          cmap=plt.cm.Blues)
        plt.show()


### Multi Class Baseline (2021 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2021.drop('Risk_Level', 1)
    X_test = test_set_2021.drop('Risk_Level', 1)
    y_train = train_set_2021['Risk_Level']
    y_test = test_set_2021['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Baseline 2021 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


### Multi Class Baseline (2020 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2020.drop('Risk_Level', 1)
    X_test = test_set_2020.drop('Risk_Level', 1)
    y_train = train_set_2020['Risk_Level']
    y_test = test_set_2020['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Baseline 2020 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


### Multi Class Baseline (2019 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2019.drop('Risk_Level', 1)
    X_test = test_set_2019.drop('Risk_Level', 1)
    y_train = train_set_2019['Risk_Level']
    y_test = test_set_2019['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Baseline 2019 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


## Weather Models:

Define Key Information:

In [ ]:
# classes to predict
classes = ['Low', 'Medium', 'High']

# set data_df = dataset
data_df = feature_set_w_truth_df

# features definition
categorical_features = [
#                        'State',
#                        'County',
#                        'Maintenance Parent Railroad Code',
#                        'Railroad Type', 
#                        'Track Type Code',
                       ]

numeric_features = [
                    'ELEV',
                    'Mean Daily Precipitation',
                    'Mean Daily Snow',
                    'Mean Daily Snow Depth',
                    'Mean Daily TempMax',
                    'Mean Daily TempMin',
                    'Max Daily Precipitation',
                    'Max Daily Snow',
                    'Max Daily Snow Depth',
                    'Max Daily TempMax',
                    'Max Daily TempMin',
                    'Min Daily Precipitation',
                    'Min Daily Snow',
                    'Min Daily Snow Depth',
                    'Min Daily TempMax',
                    'Min Daily TempMin',
                   ]

# X represents all feature columns
X = data_df.loc[:,(categorical_features + numeric_features)]
# note y should be a "Risk class" derived from # of incidents/injuries/fatalities somehow that matches 'classes' above
y = data_df['Risk_Level']

# classifiers to try
names = ["Decision Tree", "AdaBoost", "GradientBoost"]
classifiers = [DecisionTreeClassifier(max_depth=5),
               AdaBoostClassifier(),
               GradientBoostingClassifier(),
              ]

# training set ratios
tsizes = [0.7,0.8,0.9]

Transformation of Features:

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

### Multi Class Weather Model (Train/Test/Split Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )
    
    # establish empty tsize lists
    tsize_precision = []
    tsize_recall = []
    
    for tsize in tsizes:
        # print tsize being tested
        print(f'test size: {tsize}')
        # train, test, split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
        # fit model
        clf.fit(X_train, y_train)
        # get confusion matrix stats
        y_pred = clf.predict(X_test)
        N = confusion_matrix(y_test, y_pred, labels=classes)
        # note that in the above N, rows are true label, column predicted label, values indicate # of samples
        # utilize plot confusion matrix function from earlier
        print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
        print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
        print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
        plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                          normalize=True,
                          title="Weather TTS Treatment - {}".format(names[i]),
                          cmap=plt.cm.Blues)
        plt.show()


### Multi Class Weather Model (2021 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2021.drop('Risk_Level', 1)
    X_test = test_set_2021.drop('Risk_Level', 1)
    y_train = train_set_2021['Risk_Level']
    y_test = test_set_2021['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Weather 2021 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


### Multi Class Weather Model (2020 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2020.drop('Risk_Level', 1)
    X_test = test_set_2020.drop('Risk_Level', 1)
    y_train = train_set_2020['Risk_Level']
    y_test = test_set_2020['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Weather 2020 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


### Multi Class Weather Model (2019 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2019.drop('Risk_Level', 1)
    X_test = test_set_2019.drop('Risk_Level', 1)
    y_train = train_set_2019['Risk_Level']
    y_test = test_set_2019['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Weather 2019 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


## Combined Models:

Define Key Information:

In [ ]:
# classes to predict
classes = ['Low', 'Medium', 'High']

# set data_df = dataset
data_df = feature_set_w_truth_df

# features definition
categorical_features = [
                        'State',
                        'County',
                        'Maintenance Parent Railroad Code',
                        'Railroad Type', 
                        'Track Type Code',
                       ]

numeric_features = [
                    'ELEV',
                    'Mean Daily Precipitation',
                    'Mean Daily Snow',
                    'Mean Daily Snow Depth',
                    'Mean Daily TempMax',
                    'Mean Daily TempMin',
                    'Max Daily Precipitation',
                    'Max Daily Snow',
                    'Max Daily Snow Depth',
                    'Max Daily TempMax',
                    'Max Daily TempMin',
                    'Min Daily Precipitation',
                    'Min Daily Snow',
                    'Min Daily Snow Depth',
                    'Min Daily TempMax',
                    'Min Daily TempMin',
                   ]

# X represents all feature columns
X = data_df.loc[:,(categorical_features + numeric_features)]
# note y should be a "Risk class" derived from # of incidents/injuries/fatalities somehow that matches 'classes' above
y = data_df['Risk_Level']

# classifiers to try
names = ["Decision Tree", "AdaBoost", "GradientBoost"]
classifiers = [DecisionTreeClassifier(max_depth=5),
               AdaBoostClassifier(),
               GradientBoostingClassifier(),
              ]

# training set ratios
tsizes = [0.7,0.8,0.9]

Transformation of Features:

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

### Multi Class Combined Model (Train/Test/Split Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )
    
    # establish empty tsize lists
    tsize_precision = []
    tsize_recall = []
    
    for tsize in tsizes:
        # print tsize being tested
        print(f'test size: {tsize}')
        # train, test, split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
        # fit model
        clf.fit(X_train, y_train)
        # get confusion matrix stats
        y_pred = clf.predict(X_test)
        N = confusion_matrix(y_test, y_pred, labels=classes)
        # note that in the above N, rows are true label, column predicted label, values indicate # of samples
        # utilize plot confusion matrix function from earlier
        print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
        print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
        print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
        plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                          normalize=True,
                          title="Combined TTS Treatment - {}".format(names[i]),
                          cmap=plt.cm.Blues)
        plt.show()


### Multi Class Combined Model (2021 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2021.drop('Risk_Level', 1)
    X_test = test_set_2021.drop('Risk_Level', 1)
    y_train = train_set_2021['Risk_Level']
    y_test = test_set_2021['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Combined 2021 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


### Multi Class  Combined Model (2020 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2020.drop('Risk_Level', 1)
    X_test = test_set_2020.drop('Risk_Level', 1)
    y_train = train_set_2020['Risk_Level']
    y_test = test_set_2020['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Combined 2020 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()


### Multi Class Combined Model (2019 Temporal Treatment):

In [ ]:
i = -1

for classifier in classifiers:
    # print name of classifier
    i += 1
    print('classifier: {}'.format(names[i]))
    # establish pipe
    clf = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    # train, test, split
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tsize)
    X_train = train_set_2019.drop('Risk_Level', 1)
    X_test = test_set_2019.drop('Risk_Level', 1)
    y_train = train_set_2019['Risk_Level']
    y_test = test_set_2019['Risk_Level']

    # fit model
    clf.fit(X_train, y_train)
    # get confusion matrix stats
    y_pred = clf.predict(X_test)
    N = confusion_matrix(y_test, y_pred, labels=classes)
    # note that in the above N, rows are true label, column predicted label, values indicate # of samples
    # utilize plot confusion matrix function from earlier
    print('f1_score:  {}'.format(f1_score(y_test, y_pred, average="macro")))
    print('precision:  {}'.format(precision_score(y_test, y_pred, average="macro")))
    print('recall:  {}'.format(recall_score(y_test, y_pred, average="macro")))
    plot_confusion_matrix(y_test, clf.predict(X_test), classes,
                      normalize=True,
                      title="Combined 2019 Treatment - {}".format(names[i]),
                      cmap=plt.cm.Blues)
    plt.show()
